In [1]:
# Code used to create the results in section 4.7.1 of the paper 
# "Analysis and Comparison of Classification Metrics" 

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import expit, logit, logsumexp 
import sys
sys.path.append("../")
from expected_cost import ec, utils, other_metrics
from data import get_llks_for_multi_classif_task
import re


def plot_vertical_line(x, ylim, style):
    plt.plot([x,x], ylim, style)

def value_at_thr(values, thrs, sel_thr):
    # Find the value in thrs that is closest to sel_thr
    # and then return the value in values for that thr.
    i = np.argmin(np.abs(np.array(thrs)-sel_thr))
    return values[i]


**** Creating simulated data with Gaussian class distributions for 2 classes ****



In [3]:
# Get likelihoods and then convert them to LLRs, because it makes them
# nicer for plotting and for interpretation of the scores.

sim_params={'feat_std':0.15, 'score_scale':0.5, 'score_shift': [0.5,0]}
priors = [0.9, 0.1]
targets, mis_llks, cal_llks = get_llks_for_multi_classif_task('gaussian_sim', priors=priors, sim_params=sim_params)

# Get LLRs from calibrated and miscalibrated likelihoods
mis_llrs = mis_llks[:,1] - mis_llks[:,0]
cal_llrs = cal_llks[:,1] - cal_llks[:,0]

# Number of samples from each class
N0 = sum(targets==0)
N1 = sum(targets==1)
K = N0 + N1

# Plot the resulting distributions
plt.figure(figsize=(6,5))
c, hs = utils.make_hist(targets, mis_llrs)
plt.plot(c, hs[1], 'r-', label='miscalibrated scores')
plt.plot(c, hs[0], 'r:')

c, hs = utils.make_hist(targets, cal_llrs)
plt.plot(c, hs[1], 'b-', label='calibrated scores')
plt.plot(c, hs[0], 'b:')
plt.legend()

outdir = "outputs/Metrics_over_scores"
utils.mkdir_p(outdir)
plt.savefig("%s/score_dists.pdf"%outdir)


In [4]:
# Now, we take miscalibrated and calibrated llrs and choose a bunch 
# of different decision thresholds and compute a few metrics
# for each case.
score_dict = {'miscal_scores': mis_llrs, 'cal_scores': cal_llrs}

# We consider two priors, the ones in the data and the uniform one
priors_data = np.array([N0/K, N1/K])
priors_unif = np.array([0.5, 0.5])

# We consider to cost matrices, the usual 0-1 matrix
# and one with a higher weight for K10
costs_01 = ec.cost_matrix([[0, 1], [1, 0]])
costs_0b = ec.cost_matrix([[0, 1], [2, 0]])

colors = {'EC1': 'b', 'EC2': 'r', 'EC3': 'g', 'FS': 'k'} #, 'MCC': 'k'}
metrics = colors.keys()

In [6]:
for score_name, scores in score_dict.items():

    print(f"*** Metric values at different thresholds for {score_name}")

    # The dict below will accumulate a list of values for each threshold, for each metric
    metric_dict = dict([(m, []) for m in metrics])

    thrs = np.arange(-1,3,0.01)

    for thr in thrs:

        # Number of samples of class 0 with a score larger than the thr (ie, labelled as class 1)
        K01 = np.sum(scores[targets==0]>thr)
        # Number of samples of class 1 with a score smaller than the thr (ie, labelled as class 0)
        K10 = np.sum(scores[targets==1]<thr)

        # Get the normalized confusion matrix needed to compute the metrics
        R = utils.compute_R_matrix_from_counts_for_binary_classif(K01, K10, N0, N1)
        print(f"esto es ERRE {R}")

        # Compute various metrics based on the confusion matrix
        metric_dict['EC1'].append(ec.average_cost_from_confusion_matrix(R, priors_unif, costs_01, adjusted=True))
        metric_dict['EC2'].append(ec.average_cost_from_confusion_matrix(R, priors_data, costs_01, adjusted=True))
        metric_dict['EC3'].append(ec.average_cost_from_confusion_matrix(R, priors_data, costs_0b, adjusted=True))
        metric_dict['FS'].append(other_metrics.Fscore(K10, K01, N0, N1))


    for metric_name, metric_list in metric_dict.items():
        metric_dict[metric_name] = np.array(metric_list)

    plt.figure(figsize=(6,5))
    plt.plot(thrs, metric_dict['EC1'], label=r'$\mathrm{NEC}_u$', color=colors['EC1'])
    plt.plot(thrs, metric_dict['EC2'], label=r'$\mathrm{NEC}_{\beta^2=1}$', color=colors['EC2'])
    plt.plot(thrs, metric_dict['EC3'], label=r'$\mathrm{NEC}_{\beta^2=2}$', color=colors['EC3'])
    plt.plot(thrs, 1-metric_dict['FS'], label=r'$1-\mathrm{F}_{\beta=1}$', color=colors['FS'])

    # Compute the best and the Bayes (only for ECs) thresholds for each metric.
    thr_dict = dict()
    thr_dict['bayes_thr_for_EC1'] = utils.bayes_thr_for_llrs(priors_unif, costs_01)
    thr_dict['bayes_thr_for_EC2'] = utils.bayes_thr_for_llrs(priors_data, costs_01)
    thr_dict['bayes_thr_for_EC3'] = utils.bayes_thr_for_llrs(priors_data, costs_0b)
    thr_dict['best_thr_for_EC1']  = thrs[np.nanargmin(metric_dict['EC1'])]
    thr_dict['best_thr_for_EC2']  = thrs[np.nanargmin(metric_dict['EC2'])]
    thr_dict['best_thr_for_EC3']  = thrs[np.nanargmin(metric_dict['EC3'])]
    thr_dict['best_thr_for_FS']   = thrs[np.nanargmin(1-metric_dict['FS'])]

    # Plot the curves
    ylim = plt.ylim()
    for metric in metrics:
        utils.plot_vertical_line(
            thr_dict[f'best_thr_for_{metric}'], ylim, f'{colors[metric]}:'
        )
        if 'EC' in metric:
            utils.plot_vertical_line(
                thr_dict[f'bayes_thr_for_{metric}'],
                ylim,
                f'{colors[metric]}--',
            )
    plt.xlabel("Threshold")
    plt.legend(loc='upper right')
    plt.title(re.sub("cal", "calibrated", re.sub("_"," ",score_name)))
    plt.tight_layout()
    plt.savefig(f"{outdir}/{score_name}.pdf")

    # Now, print the value of every metric for every threshold above
    print("    Threshold_type       (   Thr  )    ", end='')
    for metric in metrics:
        print(f"{metric:5s}  ", end='')
    print("")

    for thr_name, thr in thr_dict.items():
        print(f"    {thr_name:<20s} ( {thr:6.3f} )   ", end='')
        for metric in metrics:
            m = value_at_thr(metric_dict[metric], thrs, thr)
            print(f"{m:5.3f}  ", end='')
        print("")
    print("")

*** Metric values at different thresholds for miscal_scores
esto es ERRE [[0.81767778 0.18232222]
 [0.046      0.954     ]]


AttributeError: module 'expected_cost.utils' has no attribute 'Fscore'

In the table above we can see that:

* The metrics are immune to calibration issues when computed for decisions made with thresholds optimized for that specific metric (e.g., the value of EC1 is the same before and after calibration for the "best_thr_for_EC1" decisions).
* With F-score we have no direct way to tell if we have a calibration problem.
* With EC, we can compare the metric for the bayes and the best threshold. If the first is much worse than 
  the latter, then we know we have a calibration problem.
* The thresholds selected for EC1 are highly suboptimal for EC2, and conversely.
* For this particular data, the decisions would be the same if we optimize the threshold for EC2 or F-score.

You can play with the priors, costs, or score distribution parameters to see how the metrics change.
Plots for all three metrics as a function of the threshold can be found in the output dir